In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf


from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
os.chdir("/content/gdrive/MyDrive/NFL_Challenge/NFL-GPT/NFL data")
os.listdir()

['train_play_prediction_categ',
 'test_play_prediction_categ',
 'train_play_prediction_binary',
 'test_play_prediction_binary',
 '.DS_Store',
 'Contact Detection',
 'Punt Prediction',
 'Analytics',
 'Impact Detection',
 'data bowl 2021',
 'data bowl 2023',
 'data bowl 2022',
 'data bowl 2020',
 'asonty',
 'Highlights_NGS_2019',
 'Highlights_NGS_Prime',
 'final_df.parquet',
 'tokens.json',
 'mapped_df.parquet',
 'train_test_split.csv',
 'class_weights.parquet',
 'checkpoint',
 'models',
 'test_tokens_NFL_GPT',
 'train_tokens_NFL_GPT',
 'training_history.csv']

In [5]:
training_data = tf.data.Dataset.load("train_play_prediction_binary")
testing_data = tf.data.Dataset.load("test_play_prediction_binary")

In [6]:
training_data

<_LoadDataset element_spec=({'def': {'token_type_ids': TensorSpec(shape=(11,), dtype=tf.int64, name=None), 'OffDef_ID': TensorSpec(shape=(11,), dtype=tf.int64, name=None), 'playId': TensorSpec(shape=(), dtype=tf.int64, name=None), 'yard_ID': TensorSpec(shape=(), dtype=tf.int64, name=None), 'position_ID': TensorSpec(shape=(11,), dtype=tf.int64, name=None), 'down_ID': TensorSpec(shape=(11,), dtype=tf.int64, name=None), 'gameId': TensorSpec(shape=(), dtype=tf.int64, name=None), 'team_ID': TensorSpec(shape=(11,), dtype=tf.int64, name=None), 'Success': TensorSpec(shape=(), dtype=tf.float64, name=None)}, 'off': {'playId': TensorSpec(shape=(), dtype=tf.int64, name=None), 'down_ID': TensorSpec(shape=(11,), dtype=tf.int64, name=None), 'position_ID': TensorSpec(shape=(11,), dtype=tf.int64, name=None), 'Success': TensorSpec(shape=(), dtype=tf.float64, name=None), 'team_ID': TensorSpec(shape=(11,), dtype=tf.int64, name=None), 'OffDef_ID': TensorSpec(shape=(11,), dtype=tf.int64, name=None), 'gameId

In [5]:
class PlayTypeEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(PlayTypeEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["PlayType"])
    return embed

class OffDefEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(OffDefEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["OffDef_ID"])
    return embed

class InputEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(InputEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["input_ids"])
    return embed

class NFLTeamEncoder(tf.keras.Model):
  def __init__(self, nb_team : int, embedding_dim : int):
        super(NFLTeamEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = nb_team,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["NFLTeam"])
    return embed

class DownEncoder(tf.keras.Model):
  def __init__(self, embedding_dim : int, nb_down = 5,):
        super(DownEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = nb_down,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["down_ID"])
    return embed

class TypeEncoder(tf.keras.Model):
  def __init__(self, vocab_size : int, embedding_dim : int):
        super(TypeEncoder, self).__init__()

        self.Embedding = tf.keras.layers.Embedding(input_dim = vocab_size,
                                                   output_dim = embedding_dim)

  def call(self, x):
    embed = self.Embedding(x["token_type_ids"])
    return embed

In [8]:
class Embedding(tf.keras.Model):
  def __init__(self,
               input_vocab_size : int,
               offdef_vocab_size : int,
               type_size: int,
               playtype_vocab_size : int,
               nfl_team_size : int,
               down_size : int,
               embedding_dim : int):
        super(Embedding, self).__init__()

        self.InputEmbedding = InputEncoder(vocab_size=input_vocab_size,
                                           embedding_dim=embedding_dim)
        self.OffDefEmbedding = OffDefEncoder(vocab_size=offdef_vocab_size,
                                             embedding_dim=embedding_dim)
        self.TypeEmbedding = TypeEncoder(vocab_size=type_size,
                                             embedding_dim=embedding_dim)
        self.PlayTypeEmbedding = PlayTypeEncoder(vocab_size=playtype_vocab_size,
                                                 embedding_dim=embedding_dim)
        self.NFLTeamEmbedding = NFLTeamEncoder(vocab_size=nfl_team_size,
                                                 embedding_dim=embedding_dim)
        self.DownEmbedding = DownEncoder(vocab_size=down_size,
                                                 embedding_dim=embedding_dim)
        self.Add = tf.keras.layers.Add()

        self.Dense = tf.keras.layers.Dense(embedding_dim, activation = "relu")

  def call(self, x):
    input_embed = self.InputEmbedding(x)
    offdef_embed = self.OffDefEmbedding(x)
    playtype_embed = self.PlayTypeEmbedding(x)
    NFLTeamEncoder_embed = self.NFLTeamEmbedding(x)
    down_embed = self.DownEmbedding(x)

    embed = self.Add([input_embed, offdef_embed, playtype_embed, NFLTeamEncoder_embed, down_embed])
    embed = self.Dense(embed)

    return embed

In [9]:
from typing import List, Optional, Union

def shape_list(tensor: Union[tf.Tensor, np.ndarray]) -> List[int]:
    """
    Deal with dynamic shape in tensorflow cleanly.

    Args:
        tensor (`tf.Tensor` or `np.ndarray`): The tensor we want the shape of.

    Returns:
        `List[int]`: The shape of the tensor as a list.
    """
    if isinstance(tensor, np.ndarray):
        return list(tensor.shape)

    dynamic = tf.shape(tensor)

    if tensor.shape == tf.TensorShape(None):
        return dynamic

    static = tensor.shape.as_list()

    return [dynamic[i] if s is None else s for i, s in enumerate(static)]

In [ ]:
class AttentionBlock(tf.keras.Model):
  def __init__(self,
               num_heads : int,
               hidden_dim : int,
               output_dim : int):
        super(AttentionBlock, self).__init__()

        self.num_attention_heads = num_heads
        self.attention_head_size = hidden_dim
        self.total_dim = num_heads * hidden_dim
        self.output_dim = output_dim

        self.Query = tf.keras.layers.Dense(self.total_dim, name = "Query")
        self.Key = tf.keras.layers.Dense(self.total_dim, name = "Key")
        self.Value = tf.keras.layers.Dense(self.total_dim, name = "Value")


        self.Dense = tf.keras.layers.Dense(output_dim, name = "Dense", activation = "relu")
        self.Add = tf.keras.layers.Add(name = "Add")
        self.Drop = tf.keras.layers.Dropout(rate = 0.1)
        self.Norm = tf.keras.layers.BatchNormalization(name = "Norm")

  def transpose_for_scores(self, tensor: tf.Tensor, batch_size: int) -> tf.Tensor:
        # Reshape from [batch_size, seq_length, all_head_size] to [batch_size, seq_length, num_attention_heads, attention_head_size]
        tensor = tf.reshape(tensor=tensor, shape=(batch_size, -1, self.num_attention_heads, self.attention_head_size))

        # Transpose the tensor from [batch_size, seq_length, num_attention_heads, attention_head_size] to [batch_size, num_attention_heads, seq_length, attention_head_size]
        return tf.transpose(tensor, perm=[0, 2, 1, 3])

  def compute_scaled_attn_scores(self, query, key):
    attention_scores = tf.matmul(query, key, transpose_b=True)  # Transpose the second sequence

    # If you want scaled dot-product attention, divide by the square root of the embedding dimension
    embedding_dim = query.shape[-1]
    scaled_attention_scores = attention_scores / tf.math.sqrt(tf.cast(embedding_dim, dtype=tf.float32))

    return scaled_attention_scores

  def compute_attention_weigths(self, query, key, temp_ids, masks):

    scaled_attn_scores = self.compute_scaled_attn_scores(query, key)

    return tf.nn.softmax(scaled_attn_scores, axis = -1)

  def get_preds_and_attention(self,
           embeddings):

    query = self.Query(embeddings)
    key = self.Key(embeddings)
    value = self.Value(embeddings)

    attention_weights = self.compute_attention_weigths(query, key)

    attention_scores = tf.matmul(attention_weights, value)
    attention_scores = self.Dense(attention_scores)

    output = self.Add([attention_scores, embeddings])
    output = self.Drop(output)
    output = self.Norm(output)
    return output, attention_weights

  def call(self,
           hidden_states : tf.Tensor,
           temporal_ids,
           attention_masks):

    batch_size = shape_list(hidden_states)[0]

    query = self.Query(hidden_states)
    queries = self.transpose_for_scores(query, batch_size)

    key = self.Key(hidden_states)
    keys = self.transpose_for_scores(key, batch_size)

    value = self.Value(hidden_states)
    values = self.transpose_for_scores(value, batch_size)

    attention_weights = self.compute_attention_weigths(queries, keys, temporal_ids, attention_masks)

    attention_scores = tf.matmul(attention_weights, values)
    attention_scores = tf.transpose(attention_scores, perm=[0, 2, 1, 3])
    attention_scores = tf.reshape(tensor=attention_scores, shape=(batch_size, -1, self.total_dim))

    attention_scores = self.Dense(attention_scores)

    output = self.Add([attention_scores, hidden_states])
    output = self.Drop(output)
    output = self.Norm(output)
    return output

class PlayPredictor(tf.keras.Model):
  def __init__(self,
               input_vocab_size : int,
               offdef_vocab_size : int,
               nfl_team_size : int,
               down_size : int,
               playtype_vocab_size : int,
               embedding_dim : int,
               hidden_dim : int):
        super(PlayPredictor, self).__init__()

        self.Embedding = Embedding(input_vocab_size =input_vocab_size,
                                   offdef_vocab_size =offdef_vocab_size,
                                   playtype_vocab_size =playtype_vocab_size,
                                   nfl_team_size =nfl_team_size,
                                   down_size = down_size,
                                   embedding_dim =embedding_dim)

        self.Attention1 = AttentionBlock(num_heads = 3,
                                         hidden_dim = hidden_dim,
                                         output_dim = embedding_dim)


        self.DenseHead = tf.keras.layers.Dense(128, activation = "gelu")
        self.Sub = tf.keras.layers.Subtract()

        self.PPHead = tf.keras.layers.Dense(1, activation = "sigmoid")

  def call(self,
           x):

    embed_off = self.Embedding(x["off"])
    h1_off = self.Attention1(embed_off)
    logits_off = self.DenseHead(h1_off)

    embed_def = self.Embedding(x["def"])
    h1_def = self.Attention1(embed_def)
    logits_def = self.DenseHead(h1_def)

    logits = self.Sub([logits_off, logits_def])
    avg_logits = tf.reduce_mean(logits, axis=1)

    pred = self.PPHead(logits)

    return pred